In [1]:
# First we load the packages
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.formula.api as smf

# I. Data Preparation

In [2]:
# Read in master data
# Note this master dataset was created in the Week 2 assignment
Master = pd.read_csv("../Data/Master.csv")
Master.head()

,Unnamed: 0,playerID,yearID,stint,G,AB,R,H,Doubles,Triples,...,lgID,salary,lnSal,debutyr,Exp,Arb,Free,POS,Catch,Infld
0,0,abbotje01,1998,1,89,244,33,68,14,1,...,AL,255000,12.449019,1997,1,0,0,OF,0,0
1,1,abbotje01,2000,1,80,215,31,59,15,1,...,NL,300000,12.611538,1997,3,1,0,OF,0,0
2,2,abbotku01,1994,1,101,345,41,86,17,3,...,NL,119000,11.686879,1993,1,0,0,SS,0,1
3,3,abbotku01,1995,1,120,420,60,107,18,7,...,NL,250000,12.429216,1993,2,0,0,SS,0,1
4,4,abbotku01,1996,1,109,320,37,81,18,7,...,NL,650000,13.384728,1993,3,1,0,SS,0,1


In [3]:
#Create Exp Squared Variable
Master['Exp2']= Master.Exp ** 2
Master['AVG']= Master.H / Master.PA
Master['ISO']= Master.SLG - Master.AVG
Master['Eye']= (Master.BB + Master.HBP)/ Master.PA

In [4]:
Master= Master[(Master.SalYear >=1995) & (Master.SalYear <=2015)]
Master= Master.drop('Unnamed: 0', axis=1)
Master.head()

,playerID,yearID,stint,G,AB,R,H,Doubles,Triples,HR,...,Exp,Arb,Free,POS,Catch,Infld,Exp2,AVG,ISO,Eye
0,abbotje01,1998,1,89,244,33,68,14,1,12,...,1,0,0,OF,0,0,1,0.261538,0.230265,0.034615
1,abbotje01,2000,1,80,215,31,59,15,1,3,...,3,1,0,OF,0,0,9,0.244813,0.150536,0.095436
2,abbotku01,1994,1,101,345,41,86,17,3,9,...,1,0,0,SS,0,1,1,0.231806,0.162397,0.056604
3,abbotku01,1995,1,120,420,60,107,18,7,17,...,2,0,0,SS,0,1,4,0.228632,0.223748,0.087607
4,abbotku01,1996,1,109,320,37,81,18,7,8,...,3,1,0,SS,0,1,9,0.232092,0.196033,0.071633


# II. Running Regressions for Each Season

In [5]:
# Write a function to run the Moneyball regression annually for free agents only
def MBExpandFA(Season):
    MB_Seas = Master[(Master.SalYear == Season) & (Master.Free == 1)]
    global lm
    lm = smf.ols(formula = 'lnSal ~ AVG + ISO + Eye + PA + Exp+ Exp2 + C(POS)', data=MB_Seas).fit()
    return;

In [6]:
# 2. Create list to store regression results
index = 0
lm_Results = [0]
for index in range(1,23):
    lm_Results.append(index)
    index = index + 1
display(lm_Results)    

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22]

In [7]:
# 3. Run regression and store results
Season = 1995
i = 0
while Season <= 2015:
    MBExpandFA(Season)
    lm_Results[i] = lm
    i = i + 1
    Season = Season + 1

In [8]:
# 4. Give each regression result a name, which is the season to which it corresponds
Season = 1995
lm_Season = ["1995"]
for Season in range(1996, 2016):
    lm_Season.append(str(Season))
    Season = Season + 1

In [9]:
#Create a list of season names to label regression results and divide list into eras
    
Pre_MB = lm_Season[:6]
MB_Period = lm_Season[6:14]
Post_MB = lm_Season[14:20]

print(Pre_MB)
print(MB_Period)
print(Post_MB)

['1995', '1996', '1997', '1998', '1999', '2000']
['2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008']
['2009', '2010', '2011', '2012', '2013', '2014']


In [10]:
# Regression results from 1994-2000
from statsmodels.iolib.summary2 import summary_col
PreMB_Out = summary_col([lm_Results[0],lm_Results[1],lm_Results[2],lm_Results[3],lm_Results[4],
                        lm_Results[5]], model_names=Pre_MB, 
                        stars=True)
print(PreMB_Out)


                1995      1996       1997       1998      1999       2000   
----------------------------------------------------------------------------
Intercept    9.9080*** 11.4819*** 12.1896*** 9.9057*** 10.1855*** 11.9028***
             (1.1403)  (1.4293)   (1.1250)   (1.1036)  (1.0077)   (0.8980)  
C(POS)[T.2B] -0.8160** -0.1368    -0.4282    -0.0241   -0.2157    -0.2705   
             (0.3754)  (0.3460)   (0.2642)   (0.2861)  (0.2293)   (0.2170)  
C(POS)[T.3B] -0.2738   -0.3603    -0.5172*   -0.0740   0.2478     -0.1153   
             (0.3050)  (0.3140)   (0.2694)   (0.2477)  (0.2180)   (0.2205)  
C(POS)[T.C]  -0.0743   0.0106     -0.2946    0.1174    -0.0485    -0.0741   
             (0.3224)  (0.3236)   (0.2634)   (0.2545)  (0.2280)   (0.2162)  
C(POS)[T.DH] -0.5598   0.3032     -0.4384    -0.2368   0.1855     -0.3203   
             (0.4076)  (0.4118)   (0.2922)   (0.2943)  (0.2426)   (0.2643)  
C(POS)[T.OF] -0.3586   0.0108     -0.0855    0.0522    0.1229     -0.0422  

In [11]:
MB_Out= summary_col([lm_Results[6], lm_Results[7], lm_Results[8], lm_Results[9], lm_Results[10], lm_Results[11], 
                    lm_Results[12],lm_Results[13]], model_names=MB_Period, stars=True)
print(MB_Out)


                2001       2002       2003      2004       2005       2006       2007       2008   
---------------------------------------------------------------------------------------------------
Intercept    11.3690*** 10.2921*** 9.9523*** 10.6775*** 10.1043*** 10.3956*** 10.8551*** 11.6809***
             (0.8410)   (1.1021)   (1.2294)  (1.1844)   (1.0212)   (1.0265)   (0.8084)   (1.0055)  
C(POS)[T.2B] 0.1695     -0.0079    -0.2705   0.0539     -0.4128    -0.1037    -0.2293    0.0265    
             (0.2200)   (0.2858)   (0.2917)  (0.2971)   (0.2584)   (0.2512)   (0.2269)   (0.2568)  
C(POS)[T.3B] 0.2337     0.2307     0.0139    -0.0219    -0.0835    0.3953*    0.1764     0.5642**  
             (0.2177)   (0.3068)   (0.2825)  (0.2969)   (0.2740)   (0.2363)   (0.2306)   (0.2342)  
C(POS)[T.C]  0.2563     0.3348     0.4833*   0.1820     -0.0549    0.2721     0.3871*    0.3589    
             (0.2245)   (0.2684)   (0.2744)  (0.2674)   (0.2430)   (0.2376)   (0.2086)   (0.2280)  

In [12]:
PostMB_Out = summary_col([lm_Results[14],lm_Results[15],lm_Results[16],lm_Results[17],lm_Results[18],
                        lm_Results[19]], model_names=Post_MB,stars=True)
print(PostMB_Out)


                2009       2010      2011      2012       2013       2014   
----------------------------------------------------------------------------
Intercept    10.5799*** 8.7517*** 9.7421*** 12.8570*** 12.8658*** 10.4102***
             (1.0237)   (1.2438)  (1.2998)  (1.1262)   (1.2939)   (1.2029)  
C(POS)[T.2B] -0.1087    0.0025    0.2004    0.1479     -0.5476**  0.3876    
             (0.3486)   (0.3402)  (0.3222)  (0.3215)   (0.2748)   (0.2569)  
C(POS)[T.3B] 0.3231     0.4618    -0.0786   0.4296     -0.2013    0.2042    
             (0.2554)   (0.3038)  (0.3070)  (0.3020)   (0.2616)   (0.2820)  
C(POS)[T.C]  0.6410**   0.1719    0.0728    0.2002     -0.2060    0.2727    
             (0.2730)   (0.3233)  (0.3467)  (0.3361)   (0.2767)   (0.2651)  
C(POS)[T.DH] 0.2849     0.1125    -0.5999*  0.4896     -0.3449    0.3179    
             (0.3481)   (0.3769)  (0.3275)  (0.3908)   (0.3284)   (0.3414)  
C(POS)[T.OF] 0.3616*    0.3795    0.1464    -0.1301    -0.3229    0.4698** 

# III. Running the Pooled Regression

In [13]:
# Create a df made up only of free agents and create a dummy variable 'PreMB'
Master_Free = Master[Master.Free == 1].copy()
Master_Free['PreMB'] = np.where(Master_Free['SalYear']<2004,1,0)
display(Master_Free[0:10])

,playerID,yearID,stint,G,AB,R,H,Doubles,Triples,HR,...,Arb,Free,POS,Catch,Infld,Exp2,AVG,ISO,Eye,PreMB
8,abbotku01,2000,1,79,157,22,34,7,1,6,...,0,1,SS,0,1,49,0.196532,0.192003,0.086705,1
17,abreubo01,2003,1,158,577,99,173,35,1,20,...,0,1,OF,0,0,49,0.248921,0.219017,0.159712,0
18,abreubo01,2004,1,159,574,118,173,47,1,30,...,0,1,OF,0,0,64,0.242637,0.300917,0.185133,0
19,abreubo01,2005,1,162,588,104,168,37,1,24,...,0,1,OF,0,0,81,0.233658,0.240832,0.171071,0
20,abreubo01,2006,3,156,548,98,163,41,2,15,...,0,1,OF,0,0,100,0.237609,0.224070,0.185131,0
21,abreubo01,2007,1,158,605,123,171,40,5,16,...,0,1,OF,0,0,121,0.244635,0.199993,0.124464,0
22,abreubo01,2008,1,156,609,100,180,39,4,20,...,0,1,OF,0,0,144,0.263158,0.208106,0.108187,0
23,abreubo01,2009,1,152,563,96,165,29,3,15,...,0,1,OF,0,0,169,0.247376,0.187792,0.142429,0
24,abreubo01,2010,1,154,573,88,146,41,1,20,...,0,1,OF,0,0,196,0.218891,0.215664,0.133433,0
25,abreubo01,2011,1,142,502,54,127,30,1,8,...,0,1,DH,0,0,225,0.217094,0.147448,0.135043,0


In [14]:
# Pooled Regression Results\n",
Pooled_lm = smf.ols(formula = 'lnSal ~ AVG + ISO + Eye + PA + Exp + Exp2 + C(POS) + PreMB*(AVG + ISO + Eye + PA + Exp + Exp2 + C(POS))', 
                    data=Master_Free).fit()
Pooled_lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  lnSal   R-squared:                       0.521
Model:                            OLS   Adj. R-squared:                  0.516
Method:                 Least Squares   F-statistic:                     122.3
Date:                Wed, 04 Jan 2023   Prob (F-statistic):               0.00
Time:                        15:26:45   Log-Likelihood:                -3313.2
No. Observations:                2841   AIC:                             6678.
Df Residuals:                    2815   BIC:                             6833.
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept             11.4301      0.307     37.267      0.000      10.829      12.031
C(POS)[T.2B]          -0.1121      0.083     -1.349      0.177      -0.275       0.051
C(POS)[T.3B]           0.1831      0.079      2.316      0.021       0.028       0.338
C(POS)[T.C]            0.0928      0.079      1.177      0.239      -0.062       0.247
C(POS)[T.DH]           0.0518      0.098      0.531      0.595      -0.139       0.243
C(POS)[T.OF]           0.1275      0.064      1.992      0.046       0.002       0.253
C(POS)[T.SS]           0.1206      0.086      1.397      0.162      -0.049       0.290
AVG                    3.5762      0.786      4.551      0.000       2.035       5.117
ISO                    1.9789      0.396      4.992      0.000       1.202       2.756
Eye                    4.3041      0.753      5.713      0.000       2.827       5.781
PA                     0.0035      0.000     26.162      0.000       0.003       0.004
Exp                    0.0961      0.041      2.352      0.019       0.016       0.176
Exp2                  -0.0044      0.002     -2.566      0.010      -0.008      -0.001
PreMB                 -0.5134      0.482     -1.065      0.287      -1.458       0.431
PreMB:C(POS)[T.2B]    -0.0953      0.127     -0.747      0.455      -0.345       0.155
PreMB:C(POS)[T.3B]    -0.2707      0.121     -2.234      0.026      -0.508      -0.033
PreMB:C(POS)[T.C]     -0.0244      0.121     -0.201      0.841      -0.262       0.214
PreMB:C(POS)[T.DH]    -0.2005      0.149     -1.346      0.178      -0.493       0.092
PreMB:C(POS)[T.OF]    -0.1567      0.097     -1.611      0.107      -0.347       0.034
PreMB:C(POS)[T.SS]    -0.0132      0.132     -0.100      0.920      -0.273       0.246
PreMB:AVG             -1.5554      1.191     -1.306      0.192      -3.891       0.780
PreMB:ISO              1.4584      0.557      2.616      0.009       0.365       2.551
PreMB:Eye             -3.2998      1.056     -3.126      0.002      -5.370      -1.230
PreMB:PA              3.8e-05      0.000      0.184      0.854      -0.000       0.000
PreMB:Exp              0.0724      0.065      1.112      0.266      -0.055       0.200
PreMB:Exp2            -0.0033      0.003     -1.170      0.242      -0.009       0.002
==============================================================================
Omnibus:                       15.371   Durbin-Watson:                   1.332
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               21.081
Skew:                           0.029   Prob(JB):                     2.64e-05
Kurtosis:                       3.418   Cond. No.                     6.08e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is corr